# Coupled Aircraft-Mission Optimization

One of the most exciting features of Aviary is the ability to formulate and solve coupled aircraft-mission design optimization problems.
Here, we mean that we are finding the optimal aircraft design parameters while simultaneously finding the optimal mission trajectory.
The reason why this is so valuable is that it enables exploration of a larger design space much more efficiently.

Solving coupled design-mission problems leads to optimal designs that would not be findable without simultaneously designing the aircraft and the trajectory.
This is especially useful for unconventional aircraft designs, operations with complex missions, and many more future-focused studies than what is commonly flying today.

This doc page builds up a coupled design problem and explains what we're doing along the way.
This process is relatively straightforward in Aviary.
We will briefly discuss the optimal results, but that is not necessarily the focus here.
Instead, we want to showcase how to do a simple coupled design study in Aviary.

You can use this as a starting point for your own exciting aircraft and mission design studies.

## Problem Definition and Explanation

We will use a conventional single-aisle commercial aircraft design as our starting point.
For all of these examples we allow the aircraft to be sized by the optimizer.
This means the gross takeoff weight is controlled to meet a mass balance.

We will perform four different optimization cases as part of this study:

- fixed mission profile, fixed aircraft wing aspect ratio
- fixed mission profile, optimized aircraft wing aspect ratio
- optimized mission profile, fixed aircraft wing aspect ratio
- optimized mission profile, optimized aircraft wing aspect ratio

We'll provide more detail individually for each case.

When we call Aviary, we will use a common `phase_info` object that we modify for each optimization case shown here:

In [ ]:
phase_info = {
    "pre_mission": {"include_takeoff": False, "optimize_mass": True},
    "climb_1": {
        "subsystem_options": {"core_aerodynamics": {"method": "computed"}},
        "user_options": {
            "optimize_mach": False,
            "optimize_altitude": False,
            "polynomial_control_order": 1,
            "num_segments": 5,
            "order": 3,
            "solve_for_distance": False,
            "initial_mach": (0.2, "unitless"),
            "final_mach": (0.72, "unitless"),
            "mach_bounds": ((0.18, 0.84), "unitless"),
            "initial_altitude": (0.0, "ft"),
            "final_altitude": (32500.0, "ft"),
            "altitude_bounds": ((0.0, 33000.0), "ft"),
            "throttle_enforcement": "path_constraint",
            "fix_initial": True,
            "constrain_final": False,
            "fix_duration": False,
            "initial_bounds": ((0.0, 0.0), "min"),
            "duration_bounds": ((35.0, 105.0), "min"),
        },
        "initial_guesses": {"times": ([0, 70], "min")},
    },
    "cruise": {
        "subsystem_options": {"core_aerodynamics": {"method": "computed"}},
        "user_options": {
            "optimize_mach": False,
            "optimize_altitude": False,
            "polynomial_control_order": 1,
            "num_segments": 5,
            "order": 3,
            "solve_for_distance": False,
            "initial_mach": (0.72, "unitless"),
            "final_mach": (0.80, "unitless"),
            "mach_bounds": ((0.7, 0.84), "unitless"),
            "initial_altitude": (32500.0, "ft"),
            "final_altitude": (36000.0, "ft"),
            "altitude_bounds": ((32000.0, 36500.0), "ft"),
            "throttle_enforcement": "boundary_constraint",
            "fix_initial": False,
            "constrain_final": False,
            "fix_duration": False,
            "initial_bounds": ((35.0, 105.0), "min"),
            "duration_bounds": ((91.5, 274.5), "min"),
        },
        "initial_guesses": {"times": ([70, 183], "min")},
    },
    "descent_1": {
        "subsystem_options": {"core_aerodynamics": {"method": "computed"}},
        "user_options": {
            "optimize_mach": False,
            "optimize_altitude": False,
            "polynomial_control_order": 1,
            "num_segments": 5,
            "order": 3,
            "solve_for_distance": False,
            "initial_mach": (0.72, "unitless"),
            "final_mach": (0.21, "unitless"),
            "mach_bounds": ((0.19, 0.84), "unitless"),
            "initial_altitude": (36000.0, "ft"),
            "final_altitude": (0.0, "ft"),
            "altitude_bounds": ((0.0, 36500.0), "ft"),
            "throttle_enforcement": "path_constraint",
            "fix_initial": False,
            "constrain_final": True,
            "fix_duration": False,
            "initial_bounds": ((126.5, 379.5), "min"),
            "duration_bounds": ((25.0, 75.0), "min"),
        },
        "initial_guesses": {"times": ([253, 50], "min")},
    },
    "post_mission": {
        "include_landing": False,
        "constrain_range": True,
        "target_range": (2080, "nmi"),
    },
}

Now, let us explain each case, formulate the Aviary problem, and optimize.
We'll discuss the results from each case and explain why they vary.

## Fixed Mission Profile, Fixed Aircraft Wing Aspect Ratio

First, let us run Aviary with a simple setup: fly a prescribed mission profile with an unchanged wing design.
Here we are varying the durations of each of the phases (climb, cruise, and descent) to minimize fuel burn across the mission.
The altitude and Mach profiles of the mission are fixed because `optimize_altitude = False` and `optimize_mach = False` for each of the phases in the `phase_info` object.

In [ ]:
from openmdao.core.problem import _clear_problem_names
_clear_problem_names()  # need to reset these to simulate separate runs
from openmdao.utils.reports_system import clear_reports
clear_reports()

In [ ]:
import aviary.api as av

aircraft_filename = 'models/test_aircraft/aircraft_for_bench_FwFm.csv'
optimizer = "IPOPT"
make_plots = True
max_iter = 200

prob = av.run_aviary(aircraft_filename, phase_info, optimizer=optimizer,
                     make_plots=make_plots, max_iter=max_iter)

Now that we've run the case successfully, let's save and print out the fuel burn value:

In [ ]:
fixed_mission_fixed_wing_fuel_burn = prob.get_val(av.Mission.Summary.FUEL_BURNED, units='kg')[0]
fixed_mission_fixed_wing_aspect_ratio = prob.get_val(av.Aircraft.Wing.ASPECT_RATIO)[0]
print('Mission fuel burn, kg:', fixed_mission_fixed_wing_fuel_burn)
print('Aspect ratio:', fixed_mission_fixed_wing_aspect_ratio)

## Fixed Mission Profile, Optimized Aircraft Wing Aspect Ratio

Now we will use the exact same `phase_info` object but set up our Aviary problem such that the aspect ratio of the wing is a design variable.
This means that Aviary is optimizing the wing aspect ratio while flying the same mission profile as above.
We would expect that by varying the wing aspect ratio, Aviary could find a lower fuel burn value.

```{note}
All of the realistic design tradeoffs associated with varying the wing aspect ratio are not necessarily captured in this problem, e.g. the wing structure would need to change. We are simply using this as an example of an aircraft design variable available in Aviary.
```

When we want to add an aircraft design variable to the Aviary problem, we need to use the Level 2 interface for Aviary.
This means we can no longer use the all-inclusive `run_aviary` function and instead need to call its constituent methods individually.
This allows us to insert a line adding the wing aspect ratio as a design variable as shown below.
This line is highlighted with an in-line comment.

In [ ]:
from openmdao.core.problem import _clear_problem_names
_clear_problem_names()  # need to reset these to simulate separate runs
from openmdao.utils.reports_system import clear_reports
clear_reports()

In [ ]:
prob = av.AviaryProblem(av.AnalysisScheme.COLLOCATION)

# Load aircraft and options data from user
# Allow for user overrides here
prob.load_inputs(aircraft_filename, phase_info)

prob.check_and_preprocess_inputs()

prob.add_pre_mission_systems()

prob.add_phases()

prob.add_post_mission_systems()

# Link phases and variables
prob.link_phases()

prob.add_driver(optimizer, max_iter=max_iter)

prob.add_design_variables()

# The following line is an example of how to add a design variable for the aspect ratio of the wing
prob.model.add_design_var(av.Aircraft.Wing.ASPECT_RATIO, lower=10., upper=14., ref=12.)

# Load optimization problem formulation
# Detail which variables the optimizer can control
prob.add_objective()

prob.setup()

prob.set_initial_guesses()

prob.run_aviary_problem(make_plots=make_plots)

In [ ]:
fixed_mission_optimized_wing_fuel_burn = prob.get_val(av.Mission.Summary.FUEL_BURNED, units='kg')[0]
fixed_mission_optimized_wing_aspect_ratio = prob.get_val(av.Aircraft.Wing.ASPECT_RATIO)[0]
print('Mission fuel burn, kg:', fixed_mission_optimized_wing_fuel_burn)
print('Aspect ratio:', fixed_mission_optimized_wing_aspect_ratio)

As expected, the optimal fuel burn value is lower for this case.
We'll discuss this in more detail after running two more cases.

## Optimized Mission Profile, Fixed Aircraft Wing Aspect Ratio

We just investigated giving the optimizer flexibility with the aircraft design while not varying the mission.
Let's now look at the results when we optimize the mission but keep the wing aspect ratio unchanged.

To do this, we will allow the optimizer to control the Mach and altitude profiles by modifying the `phase_info` object:

In [ ]:
from openmdao.core.problem import _clear_problem_names
_clear_problem_names()  # need to reset these to simulate separate runs
from openmdao.utils.reports_system import clear_reports
clear_reports()

In [ ]:
phase_info['climb_1']['user_options']['optimize_mach'] = True
phase_info['climb_1']['user_options']['optimize_altitude'] = True
phase_info['cruise']['user_options']['optimize_mach'] = True
phase_info['cruise']['user_options']['optimize_altitude'] = True
phase_info['descent_1']['user_options']['optimize_mach'] = True
phase_info['descent_1']['user_options']['optimize_altitude'] = True

prob = av.run_aviary(aircraft_filename, phase_info, optimizer=optimizer,
                     make_plots=make_plots, max_iter=max_iter)

In [ ]:
from openmdao.core.problem import _clear_problem_names
_clear_problem_names()  # need to reset these to simulate separate runs
from openmdao.utils.reports_system import clear_reports
clear_reports()

In [ ]:
phase_info['climb_1']['user_options']['optimize_mach'] = True
phase_info['climb_1']['user_options']['optimize_altitude'] = True
phase_info['cruise']['user_options']['optimize_mach'] = True
phase_info['cruise']['user_options']['optimize_altitude'] = True
phase_info['descent_1']['user_options']['optimize_mach'] = True
phase_info['descent_1']['user_options']['optimize_altitude'] = True

prob = av.run_aviary(aircraft_filename, phase_info, optimizer=optimizer,
                     make_plots=make_plots, max_iter=max_iter)

Now, let's see the fuel burn:

In [ ]:
optimized_mission_fixed_wing_fuel_burn = prob.get_val(av.Mission.Summary.FUEL_BURNED, units='kg')[0]
optimized_mission_fixed_wing_aspect_ratio = prob.get_val(av.Aircraft.Wing.ASPECT_RATIO)[0]
print('Mission fuel burn, kg:', optimized_mission_fixed_wing_fuel_burn)
print('Aspect ratio:', optimized_mission_fixed_wing_aspect_ratio)

Optimizing the mission did not have nearly as large of an impact on the fuel burn as optimizing the aspect ratio did.
However, the fuel burn still decreased.
Let us now look at the fully coupled case.

## Optimized Mission Profile, Optimized Aircraft Wing Aspect Ratio

Remember we have already modified the `phase_info` object so that the Mach and altitude profiles are optimized.
Now we return to the Level 2 way of running the problem with the wing aspect ratio as a design variable.

In [ ]:
from openmdao.core.problem import _clear_problem_names
_clear_problem_names()  # need to reset these to simulate separate runs
from openmdao.utils.reports_system import clear_reports
clear_reports()

In [ ]:
prob = av.AviaryProblem(av.AnalysisScheme.COLLOCATION)

# Load aircraft and options data from user
# Allow for user overrides here
prob.load_inputs(aircraft_filename, phase_info)

prob.check_and_preprocess_inputs()
# Preprocess inputs
prob.add_pre_mission_systems()

prob.add_phases()

prob.add_post_mission_systems()

# Link phases and variables
prob.link_phases()

prob.add_driver(optimizer, max_iter=max_iter)

prob.add_design_variables()

# prob.model.add_design_var(av.Aircraft.Engine.SCALED_SLS_THRUST, lower=25.e3, upper=30.e3, units='lbf', ref=28.e3)
prob.model.add_design_var(av.Aircraft.Wing.ASPECT_RATIO, lower=10., upper=14., ref=12.)

# Load optimization problem formulation
# Detail which variables the optimizer can control
prob.add_objective()

prob.setup()

prob.set_initial_guesses()

prob.run_aviary_problem(make_plots=make_plots)

All right, let's check out this final case's fuel burn value:

In [ ]:
optimized_mission_optimized_wing_fuel_burn = prob.get_val(av.Mission.Summary.FUEL_BURNED, units='kg')[0]
optimized_mission_optimized_wing_aspect_ratio = prob.get_val(av.Aircraft.Wing.ASPECT_RATIO)[0]
print('Mission fuel burn, kg:', optimized_mission_optimized_wing_fuel_burn)
print('Aspect ratio:', optimized_mission_optimized_wing_aspect_ratio)

Cool, it's the lowest yet!
Let's discuss these results in more detail now.

## Summary and Takeaways

We have showcased one of Aviary's most powerful capabilities here -- the ability to simultaneously design an aircraft and optimal trajectory.
By building up problem complexity, we can see how optimizing different parts of the problem lead to optimization objective improvements.

Here is a summary table of the results:

In [ ]:
import pandas as pd

# Create a dictionary with the data
data = {
    'Case': ['Fixed Mission, Fixed Wing', 'Fixed Mission, Optimized Wing', 'Optimized Mission, Fixed Wing', 'Optimized Mission, Optimized Wing'],
    'Optimize Mission': ['-', '-', '✓', '✓'],
    'Optimize Wing': ['-', '✓', '-', '✓'],
    'Aspect Ratio': [fixed_mission_fixed_wing_aspect_ratio, fixed_mission_optimized_wing_aspect_ratio, optimized_mission_fixed_wing_aspect_ratio, optimized_mission_optimized_wing_aspect_ratio],
    'Fuel Burn Value': [fixed_mission_fixed_wing_fuel_burn, fixed_mission_optimized_wing_fuel_burn, optimized_mission_fixed_wing_fuel_burn, optimized_mission_optimized_wing_fuel_burn]
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data).round(2)

# Display the DataFrame
df


We see that the fully coupled case finds the lowest fuel burn value, as expected.
In both cases where the wing aspect ratio is optimized, it moves to the higher bound.

If we didn't simultaneously design the aircraft and the mission, you would have to manually iterate by first optimizing the aircraft, then the mission, then the aircraft again, etc.
This cumbersome process is known as sequential optimization and can lead to non-optimal results for coupled systems, as detailed in Section 13.1 of the [Engineering Design Optimization textbook](https://flowlab.groups.et.byu.net/mdobook.pdf) (available for free).

Aviary is unique in its ability to solve these coupled systems using efficient gradient-based optimization.

This doc page contains a simple example, but the true power of coupled multidisciplinary optimization lies in solving more complex design problems.
We hope that you can effectively use Aviary to optimally design the next generation of exciting aircraft!
